In [3]:
from main.utils.analysis import *
%matplotlib inline

In [4]:
params = {'data_pth':'../../1/mj2非银版/data/modeling_data.csv',
    'date_colName': 'apply_date',
    'y_colName': 'credit_target',
    'org_colName':'org',
    'data_encode':'utf-8',
    'key_colNames' : ['mobile_org', 'apply_date', 'org']}
data = get_dataset(**params)

"原始数据有741764条, 根据['mobile_org', 'apply_date', 'org']去重且只保留标签列[0,1]的数据"

'去重后数据有741764条'

'credit_target, org被重命名为new_target, new_org; apply_date被格式化为new_date, new_date_ym两列'

In [5]:
data.head(1)

,mobile_org,new_date,new_org,aft_v4up2_score,new_target,i_cnt_interestLevel_one_Loan_all_30day,i_cnt_interestLevel_six_Loan_all_180day,i_cnt_interestLevel_six_Loan_all_365day,i_cnt_partner_v4_Loan_all_7day,i_cnt_partner_v4_all_LoanAssistPlat_7day,...,m_freq_night_v4_all_all_365day,m_freq_record_v4_Loan_PriOthersBank_365day,m_incr_set_recent180daypartner_v4_Loan_LoanAssistPlat_365day,m_length_last_v4_all_all_365day,m_max_freq_record_daily_v4_all_Imbank_365day,m_min_length_record_v4_Loan_all_365day,m_ratio_cnt_partner_tailplat2_v3_Loan_all_365day,m_ratio_freq_dawn_Loan_all_365day,m_ratio_freq_record_v4_Loan_LarStoBank_365day,new_date_ym
0,32210e2a7b9bbd34fd15bba19493a4e9,20240305,xk_hc,49.9037,0.0,0,2,5,7,1,...,51.0,7.0,1.0,1.4537,7.0,0.0,0.1935,0.0079,0.0059,202403


## 1. 数据清洗与变量侦测

### 1.1 去除异常月份

In [6]:
params = {
    'data': data,
    'minYmBadsample': 10,
    'minYmSample': 1000
}
data = drop_abnormal_ym(**params)

'开始分析机构样本情况'

'机构样本情况分析结束'

'输入data样本数为741764, 限制每个机构每月坏样本数>=10, 样本数>=1000'

'删去异常月份后数据大小为736933'

### 1.2 机构逐月坏样率概览

In [7]:
datasetStatis = org_analysis(data)
display(datasetStatis.head(2))
datasetStatis.to_csv("process results/datasetStatis.csv", index=False)

'开始分析机构样本情况'

'机构样本情况分析结束'

,机构,年月,单月坏样本数,单月总样本数,单月坏样率,总坏样本数,总样本数,总坏样率
0,fd_hc,202402,1207,32120,0.038,4121,123958,0.033
1,fd_hc,202403,1635,44321,0.037,4121,123958,0.033


### 1.3 变量分机构和总体坏样率概览

In [8]:
channel = {'银行': ['字节放心借', '滴滴金融'], '非银': ['xk_hc','fd_hc', '长银消金', '久恒融担（恒昌）', '分期乐欺诈',
       '宁夏海胜通'], '整体':list(data.new_org.unique())}

In [9]:
miss_org, miss_channel = missing_check(**{'data': data, 'channel': channel})
display(miss_org.head(1), miss_channel.head(1))
miss_org.to_csv('process results/miss_org.csv', index=False)
miss_channel.to_csv('process results/miss_channel.csv', index=False)

100%|██████████| 3/3 [00:23<00:00,  7.24s/it]


,机构,变量,-1111缺失率,-999缺失率,-1缺失率,nan缺失率,总缺失率
163,长银消金,i_differ_mean_interestLevel_Loan_all_90_365day,0.034,0.005,0.002,0.0,0.041


,渠道,变量,-1111缺失率,-999缺失率,-1缺失率,nan缺失率,总缺失率
68,非银,i_mom_cnt_180daypartner_v4_Loan_Imbank_360day,0.021,0.005,0.0,0.0,0.026


### 1.4 变量分机构和渠道psi概览

In [10]:
res_psi_org, res_psi_channel = detect_psi(**{'data': data, 'channel': channel})
res_psi_org.to_csv("process results/psi_org.csv", index=False)
res_psi_channel.to_csv('process results/psi_channel.csv', index=False)
display(res_psi_org.head(4), res_psi_channel.head(4))

100%|██████████| 3/3 [04:29<00:00, 81.08s/it]


,机构,变量,区间,样本数,区间psi,最大psi
1023,长银消金,m_ratio_freq_record_v4_Loan_LarStoBank_365day,Q4,10854,0.0000,0.0082
1022,长银消金,m_ratio_freq_record_v4_Loan_LarStoBank_365day,Q3,11185,0.0082,0.0082
1021,长银消金,m_ratio_freq_record_v4_Loan_LarStoBank_365day,Q2,11028,0.0033,0.0082
1020,长银消金,m_ratio_freq_record_v4_Loan_LarStoBank_365day,Q1,11034,0.0032,0.0082


,渠道,变量,区间,样本数,区间psi,最大psi
267,非银,m_ratio_freq_record_v4_Loan_LarStoBank_365day,Q4,113601,0.0000,0.3228
266,非银,m_ratio_freq_record_v4_Loan_LarStoBank_365day,Q3,117440,0.0075,0.3228
265,非银,m_ratio_freq_record_v4_Loan_LarStoBank_365day,Q2,116939,0.0011,0.3228
264,非银,m_ratio_freq_record_v4_Loan_LarStoBank_365day,Q1,116230,0.3228,0.3228


### 1.5 变量分机构和渠道和总体iv概览

In [11]:
res_iv_org, res_iv_channel = detect_iv(**{'data':data, 'method':'quantile', 'bins':5, 'channel': channel})
res_iv_org.to_csv('process results/iv_org.csv', index=False)
res_iv_channel.to_csv("process results/iv_channel.csv", index=False)
display(res_iv_org.head(2), res_iv_channel.head(2))

 67%|██████▋   | 2/3 [00:54<00:27, 27.48s/it]

'机构智汇融租下aft_v4up2_score计算iv失败, cannot do a non-empty take from an empty axes.'

100%|██████████| 3/3 [01:36<00:00, 31.74s/it]


,机构,变量,iv
308,长银消金,aft_v4up2_score,0.1590
313,长银消金,i_incr_set_recent90daypartner_v4_all_LoanAssis...,0.0621


,渠道,变量,iv
104,非银,i_cnt_slpPartner_Loan_all_90day,0.0750
108,非银,i_incr_set_recent30dayecipartner_la_Loan_all_2...,0.0671


### 1.6 去除高缺失率变量

In [12]:
data = drop_highmiss_features(**{'data':data, 'ratio': 0.3, 'cnt': 3, 'miss_org': miss_org, 'miss_channel': miss_channel})

'输入data列数为68'

'全部变量都不满足删除条件'

### 1.7去除高psi变量

In [13]:
data = drop_highpsi_features(**{'data':data, 'res_psi_org':res_psi_org, 'cnt':2, 'ratio':0.1})

'[], 在2以上机构数出现高psi'

In [14]:
def drop_lowiv_features(**kwargs) -> pd.DataFrame:
    data = kwargs.get('data').copy()
    res_iv_org = kwargs.get('res_iv_org')
    res_iv_channel = kwargs.get('res_iv_channel')
    miniv = kwargs.get('miniv')
    cnt = kwargs.get('cnt')
    
    res_iv_org['是低iv机构'] = np.where(res_iv_org['iv']<miniv, 1, 0)
    res_iv_org['低iv机构数'] = res_iv_org.groupby('变量')['是低iv机构'].transform('sum')
    drop_features = res_iv_org[res_iv_org.低iv机构数>cnt]['变量']
    keep_features = res_iv_channel[res_iv_channel.iv>=miniv]['变量']
    drop_features = list(set((set(drop_features)-set(keep_features))).intersection(data.columns))
    if len(drop_features)>0:
        display(f"去除在{cnt}上个机构iv都小于{miniv} 且在任何渠道上iv都小于{miniv}的变量, 共{len(drop_features)}个")
        data.drop(columns=drop_features, inplace=True)
    else:
        display("没有符合删去条件的变量")
    return data

In [15]:
data = drop_lowiv_features(**{'data':data, 'miniv':0.1, 'cnt':2, 'res_iv_org':res_iv_org, 'res_iv_channel': res_iv_channel})

'去除在2上个机构iv都小于0.1 且在任何渠道上iv都小于0.1的变量, 共54个'

,mobile_org,new_date,new_org,aft_v4up2_score,new_target,i_freq_record_v4_Loan_AutoFinance_180day,i_incr_set_recent30daydeviceid_v4_all_all_210day,i_incr_set_recent7daylapfpartner_la_Loan_all_180day,i_incr_set_recent90daydeviceid_v4_all_Imbank_270day,i_incr_set_recent90daypartner_v4_all_LoanAssistPlat_365day,i_max_freq_record_daily_v4_all_LoanAssistPlat_7day,i_ratio_cnt_partner_v4_Loan_AutoFinance_365day,i_ratio_freq_record_v4_Loan_AutoFinance_365day,new_date_ym
0,32210e2a7b9bbd34fd15bba19493a4e9,20240305,xk_hc,49.9037,0.0,1,0,1,0,0,1,0.0645,0.0060,202403
1,ece1d079c32e2af8e17ba008e3db0cb8,20230527,字节放心借,53.9913,0.0,0,0,0,0,0,0,0.0000,0.0000,202305
2,3c479be4d5c0f776912721098dc5ffd2,20231007,滴滴金融,54.3530,0.0,0,0,0,0,1,0,0.0000,0.0000,202310
3,72447e1a5bd41a70d373b17b8393b310,20240413,xk_hc,46.4381,0.0,1,0,0,0,4,0,0.0313,0.0103,202404
4,fc251d17e20b9d967cdd56f64d19df86,20230607,滴滴金融,49.9144,0.0,0,1,1,2,3,5,0.0000,0.0000,202306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
741759,2989771776edbc47a40b23920213e44d,20231003,智汇融租,NaN,0.0,0,0,0,0,0,0,0.0000,0.0000,202310
741760,93a8e604c3190bab6b9e338578c0ae54,20230612,智汇融租,NaN,0.0,0,0,0,0,4,0,0.0000,0.0000,202306
741761,66455d90d3c35316f60c5aaf12512a93,20230208,智汇融租,NaN,0.0,0,0,0,0,0,0,0.0000,0.0000,202302
741762,47c73e4a6ac5e5877eca7007804e8f68,20230406,智汇融租,NaN,1.0,1,0,1,0,0,0,0.0833,0.0526,202304
